In [ ]:
# Libs necessárias
def request(url:str = None,nome_arquivo:str = '', extensao:str='zip'):
    """AI is creating summary for request

    Args:
        url (str): Caminho online de onde possam ter redirecionamentos href.
        nome_arquivo (str): nome do arquivo que vamos buscar ou dos arquivos se tiverem mais de um com o mesmo nome.
        extensao (str): extenção do arquivo que vamos buscar se formos buscar por um download em específico.

    Returns:
        urls : Retorna uma lista com todos as urls que deram math com os parâmetros passados no crawler da url infomrada.
    """
    import re
    import urllib.request
    nome_arquivo = nome_arquivo.capitalize()
    print(nome_arquivo)
    abre_url = urllib.request.urlopen(url)
    
    le_url = abre_url.read()

    converte_leitura_url = le_url.decode("utf8")
    abre_url.close()
    
    urls = []
    #urls = [ f"{url}{i.replace('<', '')}" for i in re.findall(f"[\w\d]+.(?:zip|pdf|csv)<", converte_leitura_url) if i.startswith(nome_arquivo)]
    #urls = [f'{url}{i}.zip' for i in re.findall(f'href="(.*?).{extensao}', converte_leitura_url) if i.startswith(nome_arquivo)]
    urls = [ f"{url}{i}.{extensao}" for i in re.findall(f'href="(.*).(?:{extensao})"', converte_leitura_url) if i.startswith(nome_arquivo)]
    
    return urls
def download(url:str=None, salvar_onde:str= './' ):
    """

    Args:
        url (str): URL de do arquivo para download.
        salvar_onde (str): Pasta de destino do arquivo baixado.
    """
    from pySmartDL import SmartDL
    obj = SmartDL(url, salvar_onde)
    obj.start()
    path = obj.get_dest()

In [ ]:
links = ["https://dadosabertos.rfb.gov.br/CNPJ/", "http://200.152.38.155/CNPJ/"]
src = request(links[1])
print(src)

In [38]:
import os
class EXTRATOR_CNPJ: 
    import os
    def __init__(self, url:str = None,nome_arquivo:str = '', extensao:str='zip'): 
        self.url = url 
        self.nome_arquivo = nome_arquivo
        self.extensao = extensao 
        self.CNPJ_ESQUEMA =  {
                        "EMPRESAS" : {
                                "_c0" : "CNPJ_BASE",
                                "_c1" : "RAZAO_SOCIAL",
                                "_c2" : "CODIGO_NATUREZA_JURIDICA",
                                "_c3" : "CODIGO_QUALIFICACAO",
                                "_c4" : "CAPITAL_SOCIAL",
                                "_c5" : "PORTE_EMPRESA",
                                "_c6" : "ENTE_FEDERATIVO"
                                },
                        "ESTABELECIMENTOS" : {
                                "_c0" : "CNPJ_BASE",
                                "_c1" : "CNPJ_ORDEM",
                                "_c2" : "CNPJ_DV",
                                "_c3" : "MATRIZ_FILIAL",
                                "_c4" : "NOME_FANTASIA",
                                "_c5" : "SITUACAO_CADASTRAL",
                                "_c6" : "DATA_SITUACAO_CADASTRAL",
                                "_c7" : "CODIGO_MOTIVO_SITUACAO_CADASTRAL",
                                "_c8" : "CIDADE_EXTERIOR",
                                "_c9" : "CODIGO_PAIS",
                                "_c10" : "DATA_INICIO_ATIVIDADE",
                                "_c11" : "CNAE_PRINCIPAL",
                                "_c12" : "CNAE_SECUNDARIO",
                                "_c13" : "TIPO_LOGRADOURO",
                                "_c14" : "LOGRADOURO",
                                "_c15" : "NUMERO",
                                "_c16" : "COMPLEMENTO",
                                "_c17" : "BAIRRO",
                                "_c18" : "CEP",
                                "_c19" : "UF",
                                "_c20" : "CODIGO_MUNICIPIO",
                                "_c21" : "DDD_CONTATO",
                                "_c22" : "TELEFONE_CONTATO",
                                "_c23" : "DDD_COMERCIAL",
                                "_c24" : "TELEFONE_COMERCIAL",
                                "_c25" : "DDD_FAX",
                                "_c26" : "FAX",
                                "_c27" : "EMAIL",
                                "_c28" : "SITUACAO_ESPECIAL",
                                "_c29" : "DATA_SITUACAO_ESPECIAL"
                                },
                        "SIMPLES" : {
                                "_c0" : "CNPJ_BASE",
                                "_c1" : "OPCAO_SIMPLES",
                                "_c2" : "DATA_OPCAO_SIMPLES",
                                "_c3" : "DATA_EXCLUSAO_SIMPLES",
                                "_c4" : "OPCAO_MEI",
                                "_c5" : "DATA_OPCAO_MEI",
                                "_c6" : "DATA_EXCLUSAO_MEI"
                                },
                        "SOCIOS" :{
                                "_c0" : "CNPJ_BASE",
                                "_c1" : "TIPONOME",
                                "_c2" : "CPF_CNPJ",
                                "_c3" : "CODIGO_QUALIFICACAO",
                                "_c4" : "DATA_ENTRADA",
                                "_c5" : "CODIGO_PAIS",
                                "_c6" : "REPRESENTANTE",
                                "_c7" : "NOME_REPRESENTANTE",
                                "_c8" : "QUALIFICACAO_REPRESENTANTE",
                                "_c9" : "FAIXA_ETARIA"
                                },
                        "PAISES" : {
                                "_c0" : "CODIGO_PAIS", 
                                "_c1" :"NOME_PAIS"
                                },
                        "MUNICIPIOS" : {
                                "_c0" : "CODIGO_MUNICIPIO", 
                                "_c1" : "NOME_MUNICIPIO"
                                },
                        "QUALIFICACOES" : {
                                "_c0" :"CODIGO_QUALIFICACAO", 
                                "_c1" : "DESCRIÇAO_QUALIFICACAO"
                                },
                        "NATUREZAS" : {
                                "_c0" :"CODIGO_NATUREZA_JURIDICA", 
                                "_c1" : "DESCRICAO_NATUREZA_JURIDICA"
                                },
                        "MOTIVOS" : {
                                "_c0" : "CODIGO_MOTIVO_SITUACAO_CADASTRAL", 
                                "_c1" :"DESCRICAO_MOTIVO_SITUACAO_CADASTRAL"
                                },
                        "CNAES" : {
                                "_c0" : "CODIGO_CNAE",
                                "_c1" : "DESCRICAO_CNAE"
                                } 
                        }
    def request(self):
        import re
        import urllib.request
        abre_url = urllib.request.urlopen(self.url)
        le_url = abre_url.read()
        converte_leitura_url = le_url.decode("utf8")
        abre_url.close()
        urls = []
        urls =  [ f"{self.url}{i.upper()}.{self.extensao}" for i in re.findall(f'href="(.*).(?:{self.extensao})"', converte_leitura_url) if i.startswith(self.nome_arquivo)]
        return urls

    def download(self, url:str=None, destino:str = None):
        ### Com requests
        """
        import requests
        with requests.get(url, stream=True) as response:
            with open(os.path.join(salvar_onde, arquivo), 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

        """
        ### Com urllib
        """
        import urllib.request
        arquivo = url.split('/')[-1]
        # faz o download do arquivo e salva em salvar_onde/arquivo
        urllib.request.urlretrieve(url, os.path.join(salvar_onde, arquivo))
        ### Com SmartDL
        """

        ### Com SmartDL
        from pySmartDL import SmartDL

        obj = SmartDL(url, destino, threads=4, progress_bar=False)
        obj.start()
        return destino

    def run(self):
        import re
        import zipfile
        from pyspark.sql import SparkSession
        from pyspark.sql.functions import input_file_name
        urls  = self.request()
        
        # Define ou busca uma sessão do Spark
        spark = SparkSession.builder\
                .master("local[2]")\
                .appName("OnlineReader")\
                .getOrCreate()
        for url in urls:
            numero_arquivo = urls.index(url)
            # Diretório de execução do script
            current_dir = os.getcwd()
            
            # Pega o nome do arquivo pela url
            arquivo = url.split('/')[-1]
            # Define o caminho absoluto do diretório.
            salvar_onde = f"{current_dir}/RAW/{arquivo.split('.')[0]}/"

            # cria a pasta para armazenar o arquivo, se ela não existir
            if not os.path.exists(salvar_onde):
                os.makedirs(salvar_onde)
                
            # download do arquivo zip
            path = self.download(url, os.path.join(salvar_onde, arquivo))
            
            # descompactação do arquivo zip
            with zipfile.ZipFile(path, 'r') as zip_ref:

                # obtem o nome do primeiro arquivo dentro do zip
                nome_original_arquivo_zip = zip_ref.namelist()[0]

                # define um novo nome para o arquivo
                novo_nome_arquivo = f"CNPJ_{arquivo.split('.')[0]}_{numero_arquivo}.csv"
                # cria um dicionário com as informações de origem e destino
                arquivos_para_extrair = {nome_original_arquivo_zip : novo_nome_arquivo}

                # realiza a extração do arquivo zip
                zip_ref.extractall(path = f"{salvar_onde}", members=arquivos_para_extrair)
                
                # renomeia o arquivo extraído com o novo nome
                try:
                    os.rename(os.path.join(salvar_onde, nome_original_arquivo_zip), os.path.join(salvar_onde, novo_nome_arquivo))
                except Exception as e:
                    print(f"deu um pequeno erro aqui {e}") 
                    os.remove(os.path.join(salvar_onde, nome_original_arquivo_zip))
                    pass
        # leitura do arquivo CSV em um dataframe Spark
        # Obtém a lista de arquivos no diretório
        lista_arquivos = list(filter(lambda x: '.csv' in x, os.listdir(salvar_onde)))
        if len(lista_arquivos) > 1:
            csvs_files = os.path.join(salvar_onde,f"CNPJ_{arquivo.split('.')[0]}_[0-{len(lista_arquivos)}].csv")
            dados = spark.read\
                .options(delimiter = ";", header=False, inferSchema=True)\
                .csv(csvs_files)
        else:
            csv_file = os.path.join(salvar_onde, novo_nome_arquivo)
            dados = spark.read\
                .options(delimiter = ";", header=False, inferSchema=True)\
                .csv(csv_file)
        print(dados.schema)
        # USA O MÉTODO WITHCOLUMNRENAMED() PARA RENOMEAR AS COLUNAS
        for NOME_ANTIGO, NOVO_NOME in self.CNPJ_ESQUEMA[arquivo.split('.')[0]].items():
            dados = dados.withColumnRenamed(NOME_ANTIGO, NOVO_NOME)
        dados.show(1, vertical=True)
        os.remove(path)
        return dados

In [39]:
url = 'https://dadosabertos.rfb.gov.br/CNPJ/'
EC_SIMPLES = EXTRATOR_CNPJ(url=url, nome_arquivo='Simples', extensao='zip')
SIMPLES = EC_SIMPLES.run()

deu um pequeno erro aqui [WinError 183] Não é possível criar um arquivo já existente: 'c:\\Users\\ABRASEL NACIONAL\\Documents\\GitHub\\ETL_CNPJ\\TESTE_BASE/RAW/SIMPLES/F.K03200$W.SIMPLES.CSV.D30408' -> 'c:\\Users\\ABRASEL NACIONAL\\Documents\\GitHub\\ETL_CNPJ\\TESTE_BASE/RAW/SIMPLES/CNPJ_SIMPLES_0.csv'
StructType([StructField('_c0', IntegerType(), True), StructField('_c1', StringType(), True), StructField('_c2', IntegerType(), True), StructField('_c3', IntegerType(), True), StructField('_c4', StringType(), True), StructField('_c5', IntegerType(), True), StructField('_c6', IntegerType(), True)])
-RECORD 0-------------------------------------
 CNPJ_BASE             | 0                    
 OPCAO_SIMPLES         | N                    
 DATA_OPCAO_SIMPLES    | 20070701             
 DATA_EXCLUSAO_SIMPLES | 20070701             
 OPCAO_MEI             | N                    
 DATA_OPCAO_MEI        | 20090701             
 DATA_EXCLUSAO_MEI     | 20090701             
 filename             

In [40]:
print(f"Este dataframe do Simples tem {SIMPLES.count()} Observações")

Este dataframe do Simples tem 34995097 Observações


In [41]:
SIMPLES.show()

+---------+-------------+------------------+---------------------+---------+--------------+-----------------+--------------------+
|CNPJ_BASE|OPCAO_SIMPLES|DATA_OPCAO_SIMPLES|DATA_EXCLUSAO_SIMPLES|OPCAO_MEI|DATA_OPCAO_MEI|DATA_EXCLUSAO_MEI|            filename|
+---------+-------------+------------------+---------------------+---------+--------------+-----------------+--------------------+
|        0|            N|          20070701|             20070701|        N|      20090701|         20090701|file:///c:/Users/...|
|        6|            N|          20180101|             20191231|        N|             0|                0|file:///c:/Users/...|
|        8|            N|          20140101|             20211231|        N|             0|                0|file:///c:/Users/...|
|       11|            S|          20070701|                    0|        N|             0|                0|file:///c:/Users/...|
|       13|            S|          20090101|                    0|        N|       

### Salva no disco e lê após isso
``` PYTHON
# Libs
import requests # SE FOR USAR REQUESTS
import urllib.request # SE FOR USAR URLLIB
from pySmartDL import SmartDL # SE FOR USAR PYSMARTDL
import zipfile
import os
from pyspark.sql import SparkSession
current_dir = os.getcwd()

# Define ou busca uma sessão do Spark
spark = SparkSession.builder.master("local[2]") \
    .appName("OnlineReader") \
    .getOrCreate()

# Define a url de download dos dados
url = 'https://dadosabertos.rfb.gov.br/CNPJ/Simples.zip'
# Pega o nome do arquivo pela url
arquivo = url.split('/')[-1]
# Define o caminho absoluto do diretório.
salvar_onde = f"{current_dir}/RAW/{arquivo.split('.')[0]}/"

# cria a pasta para armazenar o arquivo, se ela não existir
if not os.path.exists(salvar_onde):
    os.makedirs(salvar_onde)

### Com requests
"""
    with requests.get(url, stream=True) as response:
        with open(os.path.join(salvar_onde, arquivo), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

"""
### Com urllib

# faz o download do arquivo e salva em salvar_onde/arquivo
urllib.request.urlretrieve(url, os.path.join(salvar_onde, arquivo))

### Com SmartDL
"""
    dest = os.path.join(salvar_onde, arquivo)
    obj = SmartDL(url, dest, threads=4)
    obj.start()
"""

"""
# Imprime o caminho do diretório de download
print(salvar_onde)
"""

# Descompactação do arquivo
with zipfile.ZipFile(os.path.join(salvar_onde, arquivo), 'r') as zip_ref:

    # obtem o nome do primeiro arquivo dentro do zip
    nome_original_arquivo_zip = zip_ref.namelist()[0]

    # define um novo nome para o arquivo
    novo_nome_arquivo = f"CNPJ_{arquivo.split('.')[0]}.csv"

    # cria um dicionário com as informações de origem e destino
    arquivos_para_extrair = {nome_original_arquivo_zip : novo_nome_arquivo}

    # realiza a extração do arquivo zip
    zip_ref.extractall(path = f"{salvar_onde}/", members=arquivos_para_extrair)
    
    # renomeia o arquivo extraído com o novo nome
    os.rename(os.path.join(salvar_onde, nome_original_arquivo_zip), os.path.join(salvar_onde, novo_nome_arquivo))

"""
# imprime o nome do arquivo dentro do zip e o novo nome
print(f"Arquivo dentro do zip: {nome_original_arquivo_zip}")
print(f"Novo nome do arquivo: {novo_nome_arquivo}")
"""

# Define o caminho absoluto para o arquivo
csv_file = os.path.join(salvar_onde, novo_nome_arquivo)

# Lê o arquivo em um dataframe Spark
dados = spark.read.options(delimiter=";", header=False, inferSchema=True).csv(csv_file)

# Plota primeira linha do dataframe
dados.show(1, vertical=True)
```

In [ ]:
# Libs
import requests # SE FOR USAR REQUESTS
import urllib.request # SE FOR USAR URLLIB
from pySmartDL import SmartDL # SE FOR USAR PYSMARTDL
import zipfile
import os
from pyspark.sql import SparkSession
current_dir = os.getcwd()

# Define ou busca uma sessão do Spark
spark = SparkSession.builder.master("local[2]") \
    .appName("OnlineReader") \
    .getOrCreate()

# Define a url de download dos dados
url = 'https://dadosabertos.rfb.gov.br/CNPJ/Simples.zip'
# Pega o nome do arquivo pela url
arquivo = url.split('/')[-1]
# Define o caminho absoluto do diretório.
salvar_onde = f"{current_dir}/RAW/{arquivo.split('.')[0]}/"

# cria a pasta para armazenar o arquivo, se ela não existir
if not os.path.exists(salvar_onde):
    os.makedirs(salvar_onde)

### Com requests
"""
    with requests.get(url, stream=True) as response:
        with open(os.path.join(salvar_onde, arquivo), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

"""
### Com urllib

# faz o download do arquivo e salva em salvar_onde/arquivo
urllib.request.urlretrieve(url, os.path.join(salvar_onde, arquivo))

### Com SmartDL
"""
    dest = os.path.join(salvar_onde, arquivo)
    obj = SmartDL(url, dest, threads=4)
    obj.start()
"""

"""
# Imprime o caminho do diretório de download
print(salvar_onde)
"""

# Descompactação do arquivo
with zipfile.ZipFile(os.path.join(salvar_onde, arquivo), 'r') as zip_ref:

    # obtem o nome do primeiro arquivo dentro do zip
    nome_original_arquivo_zip = zip_ref.namelist()[0]

    # define um novo nome para o arquivo
    novo_nome_arquivo = f"CNPJ_{arquivo.split('.')[0]}.csv"

    # cria um dicionário com as informações de origem e destino
    arquivos_para_extrair = {nome_original_arquivo_zip : novo_nome_arquivo}

    # realiza a extração do arquivo zip
    zip_ref.extractall(path = f"{salvar_onde}/", members=arquivos_para_extrair)
    
    # renomeia o arquivo extraído com o novo nome
    os.rename(os.path.join(salvar_onde, nome_original_arquivo_zip), os.path.join(salvar_onde, novo_nome_arquivo))

"""
# imprime o nome do arquivo dentro do zip e o novo nome
print(f"Arquivo dentro do zip: {nome_original_arquivo_zip}")
print(f"Novo nome do arquivo: {novo_nome_arquivo}")
"""

# Define o caminho absoluto para o arquivo
csv_file = os.path.join(salvar_onde, novo_nome_arquivo)

# Lê o arquivo em um dataframe Spark
dados = spark.read.options(delimiter=";", header=False, inferSchema=True).csv(csv_file)

# Plota primeira linha do dataframe
dados.show(1, vertical=True)

In [ ]:
dados.count()

In [ ]:
CNPJ_ESQUEMA =  {
        "EMPRESAS" : {
                "_c0" : "CNPJ_BASE ",
                "_c1" : "RAZAO_SOCIAL ",
                "_c2" : "CODIGO_NATUREZA_JURIDICA ",
                "_c3" : "CODIGO_QUALIFICACAO",
                "_c4" : "CAPITAL_SOCIAL ",
                "_c5" : "PORTE_EMPRESA ",
                "_c6" : "ENTE_FEDERATIVO"
                },
        "ESTABELECIMENTOS" : {
                "_c0" : "CNPJ_BASE ",
                "_c1" : "CNPJ_ORDEM ",
                "_c2" : "CNPJ_DV ",
                "_c3" : "MATRIZ_FILIAL ",
                "_c4" : "NOME_FANTASIA ",
                "_c5" : "SITUACAO_CADASTRAL",
                "_c6" : "DATA_SITUACAO_CADASTRAL ",
                "_c7" : "CODIGO_MOTIVO_SITUACAO_CADASTRAL ",
                "_c8" : "CIDADE_EXTERIOR ",
                "_c9" : "CODIGO_PAIS ",
                "_c10" : "DATA_INICIO_ATIVIDADE",
                "_c11" : "CNAE_PRINCIPAL ",
                "_c12" : "CNAE_SECUNDARIO ",
                "_c13" : "TIPO_LOGRADOURO ",
                "_c14" : "LOGRADOURO ",
                "_c15" : "NUMERO ",
                "_c16" : "COMPLEMENTO",
                "_c17" : "BAIRRO ",
                "_c18" : "CEP ",
                "_c19" : "UF ",
                "_c20" : "CODIGO_MUNICIPIO ",
                "_c21" : "DDD_CONTATO ",
                "_c22" : "TELEFONE_CONTATO ",
                "_c23" : "DDD_COMERCIAL ",
                "_c24" : "TELEFONE_COMERCIAL",
                "_c25" : "DDD_FAX ",
                "_c26" : "FAX ",
                "_c27" : "EMAIL ",
                "_c28" : "SITUACAO_ESPECIAL ",
                "_c29" : "DATA_SITUACAO_ESPECIAL"
                },
        "SIMPLES" : {
                "_c0" : "CNPJ_BASE ",
                "_c1" : "OPCAO_SIMPLES ",
                "_c2" : "DATA_OPCAO_SIMPLES ",
                "_c3" : "DATA_EXCLUSAO_SIMPLES",
                "_c4" : "OPCAO_MEI ",
                "_c5" : "DATA_OPCAO_MEI ",
                "_c6" : "DATA_EXCLUSAO_MEI"
                },
        "SOCIOS" :{
                "_c0" : "CNPJ_BASE",
                "_c1" : "TIPONOME",
                "_c2" : "CPF_CNPJ",
                "_c3" : "CODIGO_QUALIFICACAO",
                "_c4" : "DATA_ENTRADA",
                "_c5" : "CODIGO_PAIS",
                "_c6" : "REPRESENTANTE",
                "_c7" : "NOME_REPRESENTANTE",
                "_c8" : "QUALIFICACAO_REPRESENTANTE",
                "_c9" : "FAIXA_ETARIA"
                },
        "PAISES" : {
                "_c0" : "CODIGO_PAIS", 
                "_c1" :"NOME_PAIS"
                },
        "MUNICIPIOS" : {
                "_c0" : "CODIGO_MUNICIPIO", 
                "_c1" : "NOME_MUNICIPIO"
                },
        "QUALIFICACOES" : {
                "_c0" :"CODIGO_QUALIFICACAO", 
                "_c1" : "DESCRIÇAO_QUALIFICACAO"
                },
        "NATUREZAS" : {
                "_c0" :"CODIGO_NATUREZA_JURIDICA", 
                "_c1" : "DESCRICAO_NATUREZA_JURIDICA"
                },
        "MOTIVOS" : {
                "_c0" : "CODIGO_MOTIVO_SITUACAO_CADASTRAL", 
                "_c1" :"DESCRICAO_MOTIVO_SITUACAO_CADASTRAL"
                },
        "CNAES" : {
                "_c0" : "CODIGO_CNAE",
                "_c1" : "DESCRICAO_CNAE"
                } 
        }
print(CNPJ_ESQUEMA["EMPRESAS"])